### 1- Preprocessing :

define characters accepted and tashkeel accepted and remove from the training set any tashkeel and unwanted characters (eg. brackets, numbers... etc)

In [1]:
import re
import numpy as np
import pyarabic.araby as araby
import nltk
from nltk.tokenize import sent_tokenize
import tensorflow as tf
# import keras
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential,load_model
from keras.layers import Embedding, LSTM, Dense, SpatialDropout1D, Bidirectional ,Dropout, Concatenate, Flatten,Input, concatenate,TimeDistributed
from gensim.models import Word2Vec

In [2]:
arabicDictionary=['ى', 'ع', 'ظ', 'ح', 'ر', 'س', 'ي', 'ش', 'ض', 'ق', ' ', 'ث', 'ل', 'ص', 'ط', 'ك', 'آ', 'م', 'ا', 'إ', 'ه', 'ز', 'ء', 'أ', 'ف', 'ؤ', 'غ', 'ج', 'ئ', 'د', 'ة', 'خ', 'و', 'ب', 'ذ', 'ت', 'ن']
punctuations = ["،", ":", "؛", "-", "؟"]
#reading the training dataset
f = open(r"train.txt", "r",encoding="utf-8").read()
print(type(f))
print(len(f))

<class 'str'>
18515271


In [3]:
def cleanDataFromNonArabicLetters(data):
    #regex to keep arabic letters only and remove any other character (eg. brackets, numbers ...etc)
    characters_regex =r'[\s\.\u0600-\u06ff\u0750-\u077f\ufb50-\ufbc1\ufbd3-\ufd3f\ufd50-\ufd8f\ufd50-\ufd8f\ufe70-\ufefc\uFDF0-\uFDFD]+'
    processedData = re.findall(characters_regex,data)
    processedData = " ".join(processedData)
    processedData = re.sub(r"\s+"," " ,processedData) #substitute many spaces with one space only
    return processedData

In [4]:
processedData=cleanDataFromNonArabicLetters(f)
print(len(processedData))

17997403


In [5]:
def removeDiacratics(processedData):
    without_diacritics= araby.strip_diacritics(processedData)
    return without_diacritics

In [6]:
without_diacritics = removeDiacratics(processedData)
print(len(without_diacritics))

10711561


In [7]:
#take the procrssed text and separate it into sentences
def generateListsWithDiacraticsandWithout(with_diacratics, without_diacritics):
    #---------------------Preprocessing of words with diacratics------------

    #generate longStringSplited which is the list of procrssed text without brackets and numbers and dots
    #the rest of the punctuation still there
    longStringWithDecimalPoint=re.sub(r"\n", ' ', with_diacratics) #string of data with diacratics removed endlines from it
    for element in punctuations:
        longStringWithDecimalPoint=re.sub(element, '', longStringWithDecimalPoint) #remove punctuations from the string

    longStringSplited=longStringWithDecimalPoint.split('.') #split the string by dots (segment sentences) and make list of them
    longString=' '.join(longStringSplited)  #long string without decimal points

    #-------------------------------------

    #---------------------Preprocessing of words without diacratics------------
    without_diacratics_longStringWithDecimals=re.sub(r"\n", ' ', without_diacritics) #string of data without diacratics removed endlines from it
    for element in punctuations:
        without_diacratics_longStringWithDecimals=re.sub(element, '', without_diacratics_longStringWithDecimals) #remove punctuations from the string

    longStringSplited_withoutDiacratics=without_diacratics_longStringWithDecimals.split('.') #split the string by dots (segment sentences) and make list of them
    longString_withoutDiacratics=' '.join(longStringSplited_withoutDiacratics)  #long string without decimal points

    #-------------------------------------------------------------------------

    list_of_sentences=[]
    for line in longStringSplited_withoutDiacratics: #list of lists of sentences splitted in words without diacratics (used in embeddings)
        list_of_sentences.append(line.split(" "))


    # now the variable called longString has a single string with all the processed words in it
    listOfwordsWith_Diacritics=list()
    listOfwordsWith_NoDiacritics=list()

    listOfwordsWith_Diacritics=re.sub(r"\s+", ' ', longString)
    listOfwordsWith_Diacritics=listOfwordsWith_Diacritics.split(" ")

    listOfwordsWith_NoDiacritics=re.sub(r"\s+", ' ', longString_withoutDiacratics)
    listOfwordsWith_NoDiacritics=listOfwordsWith_NoDiacritics.split(" ")

    return list_of_sentences,listOfwordsWith_Diacritics,listOfwordsWith_NoDiacritics


In [8]:
list_of_sentences,listOfwordsWith_Diacritics,listOfwordsWith_NoDiacritics= generateListsWithDiacraticsandWithout(processedData, without_diacritics)

In [9]:
print(len(list_of_sentences))
print(len(listOfwordsWith_NoDiacritics))
print(len(listOfwordsWith_Diacritics))
print(listOfwordsWith_Diacritics[2101000:2101100])
print(listOfwordsWith_NoDiacritics[2101000:2101100])

40836
2102054
2102054
['أَوْ', 'سَيِّدُهَا', 'فِي', 'ذَلِكَ', 'جَازَ', 'لِأَنَّ', 'لَهُ', 'غَرَضًا', 'فِي', 'تَزَيُّنِهَا', 'لَهُ', 'كَمَا', 'فِي', 'الرَّوْضَةِ', 'وَهُوَ', 'الْأَوْجَهُ', 'وَإِنْ', 'جَرَى', 'فِي', 'التَّحْقِيقِ', 'عَلَى', 'خِلَافِ', 'ذَلِكَ', 'فِي', 'الْوَصْلِ', 'وَالْوَشْرِ', 'فَأَلْحَقَهُمَا', 'بِالْوَشْمِ', 'فِي', 'الْمَنْعِ', 'مُطْلَقًا', 'وَيُكْرَهُ', 'أَنْ', 'يَنْتِفَ', 'الشَّيْبَ', 'مِنْ', 'الْمَحَلِّ', 'الَّذِي', 'لَا', 'يُطْلَبُ', 'مِنْهُ', 'إزَالَةُ', 'شَعْرِهِ', 'وَيُسَنُّ', 'خَضْبُهُ', 'بِالْحِنَّاءِ', 'وَنَحْوِهِ', 'وَيُسَنُّ', 'لِلْمَرْأَةِ', 'الْمُزَوَّجَةِ', 'وَالْمَمْلُوكَةِ', 'خَضْبُ', 'كَفِّهَا', 'وَقَدَمِهَا', 'بِذَلِكَ', 'تَعْمِيمًا', 'لِأَنَّهُ', 'زِينَةٌ', 'وَهِيَ', 'مَطْلُوبَةٌ', 'مِنْهَا', 'لِحَلِيلِهَا', 'أَمَّا', 'النَّقْشُ', 'وَالتَّطْرِيفُ', 'فَلَا', 'يُسَنُّ', 'وَخَرَجَ', 'بِالْمُزَوَّجَةِ', 'وَالْمَمْلُوكَةِ', 'غَيْرُهُمَا', 'فَيُكْرَهُ', 'لَهُ', 'وَبِالْمَرْأَةِ', 'الرَّجُلُ', 'وَالْخُنْثَى', 'فَيَحْرُمُ', 'الْخِضَابُ', 'عَلَيْهِمَا', 'إ

In [10]:
def generateDiacraticsList(listOfwordsWith_Diacritics):
    #now that we have two separated lists we need to get the diacritics list


    listofDiacritrcs_ToWord=list()
    temp=list()
    counter=0
    for word in listOfwordsWith_Diacritics:
        while counter<len(word):
            if word[counter] in arabicDictionary: #checking if the character is a letter
                if (counter+1)<len(word):
                    #checking if the next character is also a letter, then that means that the diacritics of the current letter is none so add empty string to the list
                    if word[counter +1] in arabicDictionary:
                        temp.append("")
                        counter+=1
                        continue
                counter+=1 #if it is the end of the word (no more letters) or the next character is a diacritics -> continue looping
                continue
            else:
                if (counter+1)<len(word):
                    if word[(counter+1)] not in arabicDictionary: #if the current and the next characters are diacritics, add them together in the list
                        temp.append(word[counter]+word[counter+1])
                        counter+=2
                        continue
                temp.append(word[counter]) #if the current character only is the diacritics add it to the list
                counter+=1
        listofDiacritrcs_ToWord.append(temp.copy())
        temp.clear()
        counter=0
    return listofDiacritrcs_ToWord

In [11]:
listofDiacritrcs_ToWord= generateDiacraticsList(listOfwordsWith_Diacritics)

In [12]:
print(len(listofDiacritrcs_ToWord))
print(listOfwordsWith_Diacritics[3])
print(listOfwordsWith_NoDiacritics[3])
print(listofDiacritrcs_ToWord[3])
print(" ".join(listofDiacritrcs_ToWord[3]))
print(listofDiacritrcs_ToWord[0:100])

2102054
الْأَوَّلُ
الأول
['', 'ْ', 'َ', 'َّ', 'ُ']
 ْ َ َّ ُ
[['َ', 'ْ', 'ُ', 'ُ'], ['َ', 'ْ'], ['َ', 'َ', 'َ'], ['', 'ْ', 'َ', 'َّ', 'ُ'], ['َ', 'َ', 'ُ'], ['', 'َ', 'ْ'], ['َ', '', 'َ'], ['', '', 'َّ', 'ْ', 'َ', 'ِ', 'ُّ'], ['', 'ْ', 'ُ'], ['َ', 'َ', 'َ', 'َ'], ['َ', 'ْ', 'ُ', 'ُ'], ['ِ', 'َ', 'ْ', 'ٍ'], ['َ', 'ْ', 'َ', 'ِ', ''], ['َ', 'ِ', 'ْ', 'َ', '', 'ِ'], ['َ', 'ْ', 'ِ'], ['َ', 'ِ', '', 'ٍ'], ['ِ', '', 'ْ', 'ِ', 'ْ', 'َ', '', 'ِ'], ['ُ', 'ُ', '', 'َ'], ['َ'], ['ُ', 'ِ', 'َ'], ['ُ', 'ُ', '', 'ُ', 'ُ'], ['ِ', 'ْ'], ['', '', 'ِّ', '', 'ِ'], ['َ', 'ُ', '', 'َ', 'ً'], ['َ', 'ِ', 'ْ', 'َ', '', 'ِ'], ['ُ', 'ْ', 'َ', 'ٍ'], ['ِ', 'َ', 'َ', 'ٍ'], ['َ', 'َ', 'ِّ'], ['ُ', 'َّ', '', 'ٍ'], ['', 'ْ', 'ُ'], ['َ', 'َ', 'َ', 'َ'], ['َ', 'ْ', 'ُ'], ['', 'ْ', 'ِ'], ['َ', '', 'ٍ'], ['َ', 'ْ'], ['ِ', 'ِ', 'ْ', 'ٍ'], ['َ', 'َ', 'َ', 'َّ', 'ُ', 'ُ'], ['ُ', 'َ'], ['َ', 'ُ', 'ْ', 'ِ'], ['', '', 'ُّ', 'َّ', '', 'ِ'], ['َ', 'ِ', 'ْ', 'َ', '', 'ِ'], ['', 'ْ', 'ُ', 'ْ', 'َ', 'ِ'], ['ِ'], ['َ', 'ِ', '', 'ِ'],

2-Word Embeddings

In [13]:
def word2vecmodel(input):
    # Train Word2Vec model
    Word2Vec_model = Word2Vec(input, vector_size=100, window=5, min_count=1, workers=4, sg=0)
    Word2Vec_model.train(input,total_examples=len(listOfwordsWith_NoDiacritics),epochs=10)
    word_embeddings = {word: Word2Vec_model.wv[word] for word in Word2Vec_model.wv.index_to_key}
    return Word2Vec_model, word_embeddings

In [14]:
word_embeddings_model, word_embeddings= word2vecmodel(list_of_sentences)

3-Building the LSTM model for character level classifier

In [15]:
def formatingXandYtrain(listofDiacritrcs_ToWord,listOfwordsWith_NoDiacritics):
    for i in range(len(listofDiacritrcs_ToWord)):
        listofDiacritrcs_ToWord[i] = " ".join(listofDiacritrcs_ToWord[i])
    Y_train= np.array([listofDiacritrcs_ToWord],dtype=object).T
    X_train = np.array([listOfwordsWith_NoDiacritics],dtype=object)
    print(str(X_train.shape))
    print(str(Y_train.shape))
    return X_train,Y_train


In [16]:
X_train,Y_train= formatingXandYtrain(listofDiacritrcs_ToWord,listOfwordsWith_NoDiacritics)

(1, 2102054)
(2102054, 1)


In [18]:
def prepareForTraining(listOfwordsWith_NoDiacritics,listofDiacritrcs_ToWord):
    # Tokenize the input words and diacritics
    word_tokenizer = tf.keras.preprocessing.text.Tokenizer(char_level=True, filters='')
    word_tokenizer.fit_on_texts(listOfwordsWith_NoDiacritics)
    word_sequences = word_tokenizer.texts_to_sequences(listOfwordsWith_NoDiacritics)

    diacritic_tokenizer = tf.keras.preprocessing.text.Tokenizer(char_level=True, filters='')
    diacritic_tokenizer.fit_on_texts([''.join(d) for d in listofDiacritrcs_ToWord])
    diacritic_sequences = diacritic_tokenizer.texts_to_sequences([''.join(d) for d in listofDiacritrcs_ToWord])

    # Pad sequences to have the same length
    max_len = max(max(len(seq) for seq in word_sequences), max(len(seq) for seq in diacritic_sequences))

    return word_tokenizer,diacritic_tokenizer,max_len,word_sequences,diacritic_sequences

def padSequences(word_sequences,diacritic_sequences, max_len):
    # Pad sequences to have the same length
    padded_word_sequences = pad_sequences(word_sequences, maxlen=max_len, padding='post')
    padded_diacritic_sequences = pad_sequences(diacritic_sequences, maxlen=max_len, padding='post')
    return padded_word_sequences,padded_diacritic_sequences

def trainDiacritics(word_tokenizer,diacritic_tokenizer,max_len,padded_word_sequences,padded_diacritic_sequences):

    # Build the LSTM model
    model = Sequential()
    model.add(Embedding(input_dim=len(word_tokenizer.word_index) + 1, output_dim=50, input_length=max_len))
    model.add(LSTM(100, return_sequences=True))
    model.add(Dense(len(diacritic_tokenizer.word_index) + 1, activation='softmax'))

    # Compile the model
    model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

    # Train the model
    model.fit(padded_word_sequences, np.expand_dims(padded_diacritic_sequences, -1), epochs=10, batch_size=32)

    # Save the model for later use
    model.save("diacritic_prediction_model.h5")

    return model

# Now, you can use the trained model to predict diacritics for new Arabic words
def predict_diacritics(model,word_tokenizer, diacritic_tokenizer,max_len, word):
    word_sequence = word_tokenizer.texts_to_sequences([word])
    padded_word_sequence = pad_sequences(word_sequence, maxlen=max_len, padding='post')
    predicted_diacritic_sequence = model.predict(padded_word_sequence)
    predicted_diacritic_sequence = np.argmax(predicted_diacritic_sequence, axis=-1)
    predicted_diacritic = diacritic_tokenizer.sequences_to_texts(predicted_diacritic_sequence)
    return predicted_diacritic[0]




In [20]:
word_tokenizer,diacritic_tokenizer,max_len,word_sequences,diacritic_sequences = prepareForTraining(listOfwordsWith_NoDiacritics,listofDiacritrcs_ToWord)
padded_word_sequences,padded_diacritic_sequences=padSequences(word_sequences,diacritic_sequences,max_len)

# !!!! DO NOT RUN THE COMING CELL UNLESS YOU HAVE MADE UPDATES IN THE MODEL
you can use the model by calling:
### model = load_model('diacritic_prediction_model.h5')

In [ ]:
model=trainDiacritics(word_tokenizer,diacritic_tokenizer,max_len,padded_word_sequences,padded_diacritic_sequences)

Epoch 1/10
65690/65690 [==============================] - 919s 14ms/step - loss: 0.1176 - accuracy: 0.9577
Epoch 2/10
65690/65690 [==============================] - 926s 14ms/step - loss: 0.0918 - accuracy: 0.9656
Epoch 3/10
65690/65690 [==============================] - 15081s 230ms/step - loss: 0.0885 - accuracy: 0.9666
Epoch 4/10
65690/65690 [==============================] - 1742s 27ms/step - loss: 0.0869 - accuracy: 0.9671
Epoch 5/10
65690/65690 [==============================] - 1458s 22ms/step - loss: 0.0860 - accuracy: 0.9674
Epoch 6/10
65690/65690 [==============================] - 1331s 20ms/step - loss: 0.0853 - accuracy: 0.9676
Epoch 7/10
65690/65690 [==============================] - 1434s 22ms/step - loss: 0.0849 - accuracy: 0.9677
Epoch 8/10
65690/65690 [==============================] - 1538s 23ms/step - loss: 0.0845 - accuracy: 0.9678
Epoch 9/10
65690/65690 [==============================] - 2518s 38ms/step - loss: 0.0843 - accuracy: 0.9679
Epoch 10/10
65690/65690 [===

In [21]:
model = load_model('./models/diacritic_prediction_model.h5')

In [23]:
# Example usage
# input = "الذي علم بالقلم"
input= "ذهب علي إلى الشاطئ"
# input = "قال الزركشي ابن عرفة قوله بلفظ يقتضيه كإنكار غير حديث بالإسلام وجوب ما علم وجوبه من الدين ضرورة كإلقاء مصحف بقذر"
input_word = input.split(' ')
word_vectors = [word_embeddings_model.wv[word] for word in input_word if word in word_embeddings_model.wv]
# print(word_vectors)
outputString=""
print("number of words",len(input_word))
for word in input_word:
  predicted_diacritics1 = predict_diacritics(model,word_tokenizer, diacritic_tokenizer,max_len, word)
  #print(predicted_diacritics1.split(' '))
  predictedDiacritics=(predicted_diacritics1.split('  '))
  temp=list()
  for diacrtic in predictedDiacritics:
    # print(diacrtic)
    # print(len(diacrtic))
    if ' ' in diacrtic:
      # print("after space removel")
      diacrtic=diacrtic.replace(' ','')
      # print(diacrtic)
      # print(len(diacrtic))
      temp.append(diacrtic)
      continue
    temp.append(diacrtic)
  predictedDiacritics=temp.copy()
  tempString=""
  for i in range(len(word)):
    tempString+=word[i]
    if i<len(predictedDiacritics):
      tempString+=predictedDiacritics[i]
  print(tempString)
  tempString+=" "
  outputString+=tempString

print(outputString)


number of words 4
1/1 [==============================] - 1s 527ms/step
ذَهَبَ
1/1 [==============================] - 0s 18ms/step
عَلَيَّ
1/1 [==============================] - 0s 18ms/step
إلَى
1/1 [==============================] - 0s 25ms/step
الَّْشاِطِئ
ذَهَبَ عَلَيَّ إلَى الَّْشاِطِئ 


In [24]:
def get_word_context_embedding(word):
    try:
        return word_embeddings_model.wv[word]
    except KeyError:
        return np.zeros(word_embeddings_model.vector_size)  # Return zero vector for out-of-vocabulary words

word_context_embeddings = [get_word_context_embedding(word) for word in listOfwordsWith_NoDiacritics]
word_context_embeddings_padded = pad_sequences(word_context_embeddings, maxlen=max_len, padding='post')

In [25]:
print(word_context_embeddings_padded.shape)
print(padded_word_sequences.shape)

(2102054, 24)
(2102054, 24)


In [26]:
combined_embeddings = np.concatenate([word_context_embeddings_padded, padded_word_sequences], axis=-1)
max_len_diacritic = combined_embeddings.shape[1]  # Adjust to match the expected model input length
print(combined_embeddings.shape)

# padded_word_sequences = pad_sequences(word_sequences, maxlen=max_len, padding='post')
padded_diacritic_sequences = pad_sequences(diacritic_sequences, maxlen=max_len_diacritic, padding='post')

(2102054, 48)


In [ ]:
# # Modify the LSTM Model
# model = Sequential()
# model.add(Embedding(input_dim=len(word_tokenizer.word_index) + 1, output_dim=200, input_length=max_len_diacritic))
# model.add(Bidirectional(LSTM(100, return_sequences=True)))
# model.add(Bidirectional(LSTM(100, return_sequences=True)))
# model.add(Dense(len(diacritic_tokenizer.word_index) + 1, activation='softmax'))

# # Compile and Train the Model
# model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# max_index = len(diacritic_tokenizer.word_index)  # Maximum index in the vocabulary
# if np.max(padded_diacritic_sequences) > max_index or np.min(padded_diacritic_sequences) < 0:
#     # Adjust indices if they are out of range
#     padded_diacritic_sequences = np.clip(padded_diacritic_sequences, 0, max_index)

# # Reshape y_true to match the model's expected output shape
# # Expand dimensions for the sparse_categorical_crossentropy
# y_true = np.expand_dims(padded_diacritic_sequences, -1)

# # Train the model
# print("combined_embeddings shape:", combined_embeddings.shape)
# print("y_true shape:", y_true.shape)

# print("Word tokenizer vocab size:", len(word_tokenizer.word_index))
# print("Diacritic tokenizer vocab size:", len(diacritic_tokenizer.word_index))

# model.fit(combined_embeddings, y_true, epochs=1, batch_size=64)


# # Save the model for later use
# model.save("diacritic_prediction_model_with_word2vec.h5")

In [ ]:
# model2=trainDiacritics(word_tokenizer,diacritic_tokenizer,max_len,word_sequences,diacritic_sequences,padded_word_sequences,padded_diacritic_sequences)